In [ ]:
import os
import random
import numpy as np
import pickle as pkl
# import networkx as nx
import scipy.sparse as sp
from math import log
from sklearn import svm
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
import re
import sys
sys.path.append('../')
#from utils.utils import clean_str, loadWord2Vec  
import sys
import pandas as pd
import numpy as np
import csv
import torch

sys.path.append('../')


In [ ]:
!nvidia-smi

In [ ]:
conda install -c dglteam dgl-cuda10.0

In [ ]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import sys

In [ ]:
datasets = ['20ng', 'r8', 'R52', 'ohsumed', 'mr']
# build corpus
dataset = datasets[1]

In [ ]:
total_docs = []
train_data = []
test_data = []
train_docs = []
test_docs = []
label=[]
train_label=[]
test_label=[]

In [ ]:

with open('./data/' + dataset + '.train.txt', 'r', errors='ignore') as f:
    lines = f.readlines()
    for line in lines:
        train_data.append(line.strip())
        temp = line.split("\t")
        train_label.append(temp[0])
        label.append(temp[0])
        train_docs.append(temp[1])
        total_docs.append(temp[1])
        

In [ ]:

with open('./data/' + dataset + '.test.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        test_data.append(line.strip())
        temp = line.split("\t")
        test_label.append(temp[0])
        label.append(temp[0])
        test_docs.append(temp[1])
        total_docs.append(temp[1])


In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stop_words)

In [ ]:

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [ ]:
word_freq = {}  # to remove rare words

for line in total_docs:
    temp = clean_str(line)
    words = temp.split()
    for word in words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1


In [ ]:
len(word_freq)

In [ ]:
clean_doc = []
for line in total_docs:
    temp = clean_str(line)
    words = temp.split()
    doc_words = []
    for word in words:
        # word not in stop_words and word_freq[word] >= 5
        if dataset == 'mr':
            doc_words.append(word)
        elif word not in stop_words and word_freq[word] >= 5:
            doc_words.append(word)

    doc_str = ' '.join(doc_words).strip()
    clean_doc.append(doc_str)
    
clean_corpus_str = '\n'.join(clean_doc)


In [ ]:
clean_corpus_str

In [ ]:
with open('./data/' + dataset + '.clean.txt', 'w') as f:
    f.write(clean_corpus_str)

In [ ]:
min_len = 10000
aver_len = 0
max_len = 0 
with open('./data/' + dataset + '.clean.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        temp = line.split()
        aver_len = aver_len + len(temp)
        if len(temp) < min_len:
            min_len = len(temp)
        if len(temp) > max_len:
            max_len = len(temp)

aver_len = 1.0 * aver_len / len(lines)
print('Min_len : ' + str(min_len))
print('Max_len : ' + str(max_len))
print('Average_len : ' + str(aver_len))


In [ ]:
train_ids = list(range(len(train_label)))
test_ids = list(range(len(train_ids),len(label)))

In [ ]:
word_freq = {}
word_set = set()
with open('./data/' + dataset + '.clean.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        temp = line.split()
        for word in temp:
            word_set.add(word)
            if word in word_freq:
                word_freq[word] += 1
            else:
                word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

In [ ]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 
  
# Main Function 
word_ids = my_dictionary() 

In [ ]:
for i in range(len(vocab)):
    word_ids.add(i, vocab[i])     
len(word_ids)

In [ ]:
s=len(train_ids)+len(word_ids)

In [ ]:
adj_doc_word=np.zeros([s,s])

In [ ]:
adj_doc_word.shape

In [ ]:
clean_corpus_str

In [ ]:
corpus=[]
with open('./data/' + dataset + '.clean.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        temp = line.split()
        doc_str = ' '.join(temp).strip()
        corpus.append(doc_str)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())
print(X.shape)

In [ ]:
doc_word_tfidf=X.toarray()

In [ ]:
# word co-occurence with context windows
window_size = 20
windows = []

for doc_words in corpus:
    words = doc_words.split()
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)

In [ ]:
word_window_freq = {}
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])

In [ ]:
word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

In [ ]:
len(word_id_map)

In [ ]:
word_pair_count = {}
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i]
            word_i_id = word_id_map[word_i]
            word_j = window[j]
            word_j_id = word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1

In [ ]:
word_pair_count

In [ ]:

row = []
col = []
weight = []

# pmi as weights

num_window = len(windows)

In [ ]:
wordtowords_pmi=np.identity(vocab_size)

In [ ]:
wordtowords_pmi.shape

In [ ]:
for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    wordtowords_pmi[i][j]=pmi
    wordtowords_pmi[j][i]=pmi
    
    


In [ ]:
edge_weight = np.concatenate((doc_word_tfidf, wordtowords_pmi), axis=0)
edge_mat= np.concatenate((doc_word_tfidf, wordtowords_pmi), axis=0)


In [ ]:
edge_mat[edge_mat > 0] = 1

In [ ]:
tot_nodes=edge_mat.shape[0]

In [ ]:
num_docs=len(label)

In [ ]:
edge=[]
p=[]
for i in range(tot_nodes):
    for j in range(vocab_size):
        if(edge_mat[i][j]==1):
            p.append(i)
            p.append(j+num_docs)
            edge.append(p)
            p=[]
            

In [ ]:
edge

In [ ]:
#adding self edge
for i in range(num_docs):
    p=[]
    p.append(i)
    p.append(i)
    edge.append(p)

In [ ]:
len(edge)

In [ ]:
edge[0][1]

In [ ]:
features=np.identity(tot_nodes)
features.shape

In [ ]:
src_idx=[]
dst_idx=[]
for i in range(len(edge)):
    src_idx.append(edge[i][0])
    dst_idx.append(edge[i][1])

In [ ]:
src = np.asarray(src_idx)
dst = np.asarray(dst_idx)

In [ ]:
g = dgl.DGLGraph((src,dst))
print(g.node_attr_schemes())
g

In [ ]:
for i in range(vocab_size):
    label.append(0)

In [ ]:
len(label)==tot_nodes

In [ ]:
s=np.zeros(tot_nodes)
for i in train_ids:
    s[i]=1
for i in range(7674,15017):
    s[i]=1
train_mask=(s>0)
train_mask

In [ ]:
s=np.zeros(tot_nodes)
for i in test_ids:
    s[i]=1
for i in range(7674,15017):
    s[i]=1
test_mask=(s>0)
test_mask

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
output = integer_encoded.reshape(len(integer_encoded), 1)
#output = onehot_encoder.fit_transform(integer_encoded)
#print(onehot_encoded)

In [ ]:
out=len(np.unique(output))

In [ ]:
!nvidia-smi

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

In [ ]:
features= torch.from_numpy(features)
g_feat=features
g_feat=g_feat.to(device)
g_feat=g_feat.type(torch.FloatTensor)

In [ ]:
type(g_feat)

In [ ]:
output= torch.from_numpy(output)
output=output.to(device)
output=output.type(torch.LongTensor)


In [ ]:
tot_nodes

In [ ]:
output=output.reshape([tot_nodes])

In [ ]:
# embed_feature_size is the output dim of first layer
embed_feature_size=16
fesize=g_feat.shape[1]


In [ ]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

In [ ]:
acc_train_matrix=np.zeros(35)
acc_test_matrix=np.zeros(35)

In [ ]:
Jumping Knowledge

In [ ]:
import dgl.function as fn
import torch

AGGREGATIONS = {
    'sum': torch.sum,
    'mean': torch.mean,
    'max': torch.max,
}


class GraphConvLayer(torch.nn.Module):
    """Graph convolution layer.

    Args:
        in_features (int): Size of each input node.
        out_features (int): Size of each output node.
        aggregation (str): 'sum', 'mean' or 'max'.
                           Specify the way to aggregate the neighbourhoods.
    """
    def __init__(self, in_features, out_features, aggregation='sum'):
        super(GraphConvLayer, self).__init__()

        if aggregation not in AGGREGATIONS.keys():
            raise ValueError("'aggregation' argument has to be one of "
                             "'sum', 'mean' or 'max'.")
        self.aggregate = lambda nodes: AGGREGATIONS[aggregation](nodes, dim=1)

        self.linear = torch.nn.Linear(in_features, out_features)
        self.self_loop_w = torch.nn.Linear(in_features, out_features)
        self.bias = torch.nn.Parameter(torch.zeros(out_features))

    def forward(self, graph, x):
        graph.ndata['h'] = x
        graph.update_all(
            fn.copy_src(src='h', out='msg'),
            lambda nodes: {'h': self.aggregate(nodes.mailbox['msg'])})
        h = graph.ndata.pop('h')
        h = self.linear(h)
        return h + self.self_loop_w(x) + self.bias

In [ ]:
import torch
import torch.nn.functional as F

#from .graph_conv_layer import GraphConvLayer


class JKNetConcat(torch.nn.Module):
    """An implementation of Jumping Knowledge Network (arxiv 1806.03536) which
    combine layers with concatenation.

    Args:
        in_features (int): Size of each input node.
        out_features (int): Size of each output node.
        n_layers (int): Number of the convolution layers.
        n_units (int): Size of the middle layers.
        aggregation (str): 'sum', 'mean' or 'max'.
                           Specify the way to aggregate the neighbourhoods.
    """
    def __init__(self, in_features, out_features, n_layers=6, n_units=16,
                 aggregation='sum'):
        super(JKNetConcat, self).__init__()
        self.n_layers = n_layers

        self.gconv0 = GraphConvLayer(in_features, n_units, aggregation)
        self.dropout0 = torch.nn.Dropout(0.5)
        for i in range(1, self.n_layers):
            setattr(self, 'gconv{}'.format(i),
                    GraphConvLayer(n_units, n_units, aggregation))
            setattr(self, 'dropout{}'.format(i), torch.nn.Dropout(0.5))
        self.last_linear = torch.nn.Linear(n_layers * n_units, out_features)

    def forward(self, graph, x):
        layer_outputs = []
        for i in range(self.n_layers):
            dropout = getattr(self, 'dropout{}'.format(i))
            gconv = getattr(self, 'gconv{}'.format(i))
            x = dropout(F.relu(gconv(graph, x)))
            layer_outputs.append(x)

        h = torch.cat(layer_outputs, dim=1)
        return self.last_linear(h)


In [ ]:
import torch
import torch.nn.functional as F

#from .graph_conv_layer import GraphConvLayer


class JKNetMaxpool(torch.nn.Module):
    """An implementation of Jumping Knowledge Network (arxiv 1806.03536) which
    combine layers with Maxpool.

    Args:
        in_features (int): Size of each input node.
        out_features (int): Size of each output node.
        n_layers (int): Number of the convolution layers.
        n_units (int): Size of the middle layers.
        aggregation (str): 'sum', 'mean' or 'max'.
                           Specify the way to aggregate the neighbourhoods.
    """
    def __init__(self, in_features, out_features, n_layers=6, n_units=16,
                 aggregation='sum'):
        super(JKNetMaxpool, self).__init__()
        self.n_layers = n_layers

        self.gconv0 = GraphConvLayer(in_features, n_units, aggregation)
        self.dropout0 = torch.nn.Dropout(0.5)
        for i in range(1, self.n_layers):
            setattr(self, 'gconv{}'.format(i),
                    GraphConvLayer(n_units, n_units, aggregation))
            setattr(self, 'dropout{}'.format(i), torch.nn.Dropout(0.5))
        self.last_linear = torch.nn.Linear(n_units, out_features)

    def forward(self, graph, x):
        layer_outputs = []
        for i in range(self.n_layers):
            dropout = getattr(self, 'dropout{}'.format(i))
            gconv = getattr(self, 'gconv{}'.format(i))
            x = dropout(F.relu(gconv(graph, x)))
            layer_outputs.append(x)

        h = torch.stack(layer_outputs, dim=0)
        h = torch.max(h, dim=0)[0]
        return self.last_linear(h)


In [ ]:
#  crucial Hyperparameters
#n_layers=number of layers in network
#n_units=number of units in network
#node_aggregation= ['sum','mean','max']
n_layers=2
n_units=128
node_aggregation='sum'

In [ ]:
device

In [ ]:
# just uncommnet the model you want to use currently to JKNET CONCAT

model = JKNetConcat(fesize, out, n_layers,n_units,node_aggregation).to(device)
#model = JKNetMaxpool(fesize, out, n_layers,n_units,node_aggregation).to(device)

In [ ]:
print(model)

In [ ]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph


In [ ]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
acc_train_matrix=np.zeros(50)
acc_test_matrix=np.zeros(50)

In [ ]:
import time
import numpy as np

optimizer = th.optim.Adam(net.parameters(),  lr=0.03, weight_decay=0.05)
dur = []
for epoch in range(45):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = model(g, g_feat)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], output[train_mask])
    Train_acc = evaluate(net, g, g_feat, output, train_mask)
    print("Epoch {:05d} | Loss {:.4f} | Train Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), Train_acc, np.mean(dur)))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    Test_acc = evaluate(net, g, g_feat, output, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), Test_acc, np.mean(dur)))
    train_acc=(((torch.from_numpy(np.array(Train_acc, dtype=np.float64)))))
    test_acc=(((torch.from_numpy(np.array(Test_acc, dtype=np.float64)))))

    acc_train_matrix[epoch]=train_acc
    acc_test_matrix[epoch]=test_acc
"""
    if(epoch>5 and acc_train_matrix[epoch]-acc_train_matrix[epoch-1]<0.0005 and 
       acc_test_matrix[epoch]-acc_test_matrix[epoch-1]<0.0005):
        break
    """